# Optimizando el Diseño de la Ciclovía de la Ciudad de México con un Algoritmo tipo Metropolis-MonteCarlo 

## El análogo al Modelo de Ising

### Definiciones básicas

***
#### Los estados posibles por calle
El algoritmo de Metropolis es ampliamente aplicado en simulaciones computacionales del modelo de Ising, en el que se estudia un sistema físico que consta de partículas dispuestas en un *lattice* cuyas interacciones entre sí mismas y con un campo externo determinan su estado (que toma los valores de +1 o -1) y por ende el estado global del sistema dada una temperatura.

Mi objetivo en este trabajo es proponer un algoritmo de Metropolis que permita optimizar el diseño de las ciclovías de la Ciudad de México dado un presupuesto, un campo de necesidades y los diferentes estados que puede tomar cada calle dentro de una red. Primero es necesario crear el objeto computacional con el que voy a tratar, que es una red de calles con distintos estados posibles. Los estados que voy a considerar son

| spin | ID  | cap | att | características del carril                  |
|------|-----|-----|-----|---------------------------------------------|
| 14   | CTR | 4   | 7   | calle cetram                                |
| 13   | ESS | 3   | 6   | exclusivo con ecobici y biciestacionamiento |
| 12   | ESN | 3   | 5   | exclusivo con ecobici                       |
| 11   | ENS | 3   | 4   | exclusivo con biciestacionamiento           |
| 10   | ENN | 3   | 3   | exclusivo sin estaciones                    |
| 9    | BSS | 2   | 5   | bus-bici con ecobici y biciestacionamiento  |
| 8    | BSN | 2   | 4   | bus-bici con ecobici                        |
| 7    | BNS | 2   | 3   | bus-bici con biciestacionamiento            |
| 6    | BNN | 2   | 2   | bus-bici sin estaciones                     |
| 5    | MSS | 1   | 4   | mixto con ecobici y biciestacionamiento     |
| 4    | MSN | 1   | 3   | mixto con ecobici                           |
| 3    | MNS | 1   | 2   | mixto con biciestacionamiento               |
| 2    | MNN | 1   | 1   | mixto sin estaciones                        |
| 1    | NNN | 0   | 0   | sin ciclovía                                |

Los spines más altos son los que representan mayor energía (los que mayor presupuesto consumen). Cada uno de los estados posibles debe tener asociado un costo de constrcción (que debe ser función de su longitud y de sus características), un factor de interacción con la red, que está a asociado a la conectividad (para evitar desconexiones) y un factor de interacción con las necesidades de ciclovía locales, que actuarán como *campo externo*.

In [ ]:
struct streetState
    # estos valores son fijos
    id::String # identificador de un estado
    # estos valores se utilizarán para calcular las interacciones entre estados y con las necesidades
    spin::Int # el spin del estado
    cap::Int # la capacidad de diseño de un estado
    attPot::Int # la capacidad de acceso de usuarios de un estado
    # estos valores se calculan
    cost::Float64 # el costo de un estado
    intField::Float64 # el valor de interacción de un estado con las necesidades
    intNet::Float64 # el valor de interacción de un estado con la red
end

list_states = [[1,"NNN",0,0],
               [2,"MNN",1,1],
               [3,"MNS",1,2],
               [4,"MSN",1,3],
               [5,"MSS",1,4],
               [6,"BNN",2,2],
               [7,"BNS",2,3],
               [8,"BSN",2,4],
               [9,"BSS",2,5],
               [10,"ENN",3,3],
               [11,"ENS",3,4],
               [12,"ESN",3,5],
               [13,"ESS",3,6],
               [14,"MSS",4,7]]

function setstate!(s::streetState,x::Int) 
    s.id = list_states[x][2]
    s.spin = list_states[x][1]
    s.cap = list_states[x][3]
    s.attPot = list_states[x][4]
end
    
setcost!(s::streetState, x::Float64) = s.cost = x
setifl!(s::streetState, x::Float64) = s.intField = x
setint!(s::streetState, x::Float64) = s.intNet = x

***
#### La red de calles
Además hacen falta las coordenadas que van a caracterizar cada una de las calles de la red y la lista de vecinos. En la siguiente celda defino las dos estructuras básicas.

In [ ]:
struct streetNetwork
    coords::Vector{NTuple{2,Float64}} # la lista de coordenadas características de cada calle en la red 
    neighbors::Vector{Vector{Int}} # la lista de vecinos de la red de calles
    states::Vector{streetState} # la lista de estados de las calles de la red
end

getstatei(sn::streetNetwork,i::int) = sn.states[i] 
setstatei!(sn::streetNetwork,i::int,s::streetState) = sn.states[i] = s

### Calculos básicos por estado, el análogo del campo

In [ ]:
import Base.copy

function copy(l::Lattice)
    return Lattice(copy(l.state), copy(l.spins), l.neighbours, l.coords)
end
function Lattice(m::Int, n::Int, g::Function, J::Float64, h::Float64)
    nspins = m*n
    s = [-1,1]
    spins = [ rand(s) for i in 1:nspins ]
    neighbours,coords = g(m,n)

    H = 0; M = 0
    for i in 1:nspins
        for j in neighbours[i]
            if j < i; continue; end # repeated pairs.
            H += spins[i] * spins[j]
        end
        M += spins[i]
    end
    H = -J*H - h*M # Energy
    M = M/nspins # Magnetization
    return Lattice([H,M],spins,neighbours,coords)
end